In [1]:
!apt-get install git
!git clone https://github.com/m-faizan-mahmood/infinitnews.ai.git
!git status
!git --version
!pwd

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Cloning into 'infinitnews.ai'...
remote: Enumerating objects: 168, done.
remote: Counting objects: 100% (168/168), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 168 (delta 80), reused 148 (delta 69), pack-reused 0 (from 0)
Receiving objects: 100% (168/168), 9.77 MiB | 25.51 MiB/s, done.
Resolving deltas: 100% (80/80), done.
fatal: not a git repository (or any of the parent directories): .git
git version 2.34.1
/content


In [4]:
!pip install pyttsx3

Processing article: https://tribune.com.pk/story/2511829/air-pollution-and-its-impact-on-fertility-in-men-and-women
Processing article: https://tribune.com.pk/story/2511828/jung-woo-sung-confirms-paternity-of-moon-ga-bis-child-amid-public-scrutiny


In [1]:
!pip install gTTS

In [3]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 2.7 MB/s eta 0:00:00


In [8]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 59.7 MB/s eta 0:00:00


In [10]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.4 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


# Data Scraping

In [2]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import os
import threading

# Base URL of the website
base_url = "https://tribune.com.pk/latest"

# Function to extract articles from a single page
def extract_articles(page_url):
    try:
        response = requests.get(page_url)
        response.raise_for_status()  # Check for request errors
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract all articles with their links and titles
        articles = []
        for item in soup.find_all('h2', class_='title-heading add-mrgn-top lh-mb'):
            title = item.get_text(strip=True)
            link = item.find_previous('a', href=True)['href'] if item.find_previous('a', href=True) else "Link not found"
            articles.append({'name': title, 'article site': link})

        return articles

    except Exception as e:
        print(f"Error extracting articles: {e}")
        return []

# Function to save articles into a CSV file
def save_to_csv(articles, filename):
    with open(filename, 'a', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['name', 'article site']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        if csvfile.tell() == 0:  # Write header if the file is empty
            writer.writeheader()
        writer.writerows(articles)

# Function to extract details from a single article
def extract_article_details(url):
    try:
        # Send a GET request to the article URL
        response = requests.get(url)
        response.raise_for_status()  # Check for request errors

        # Parse the webpage content
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract the headline
        headline = soup.find('h1').get_text(strip=True) if soup.find('h1') else "Headline not found"

        # Extract the publication date (only the date part)
        date = soup.find('div', {'class': 'left-authorbox'}).find_all('span')[-1].get_text(strip=True) if soup.find('div', {'class': 'left-authorbox'}) else "Date not found"

        # Extract the main content of the article
        content = " ".join([p.get_text(strip=True) for p in soup.find_all('p')]) if soup.find_all('p') else "Content not found"

        return {'url': url, 'headline': headline, 'date': date, 'content': content}

    except Exception as e:
        print(f"Error processing URL {url}: {e}")
        return {'url': url, 'headline': "Error", 'date': "Error", 'content': "Error"}

# Function to process each URL from the CSV file
def process_articles_from_csv(input_csv, output_csv, processed_urls):
    with open(input_csv, 'r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        urls = [row['article site'] for row in reader if row['article site'] not in processed_urls]

    # For each URL, extract details, save them, and update the processed URLs
    for url in urls:
        print(f"Processing article: {url}")
        article_details = extract_article_details(url)

        # Save the article details in the output CSV
        with open(output_csv, 'a', newline='', encoding='utf-8') as csvfile:
            fieldnames = ['url', 'headline', 'date', 'content']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            if os.path.getsize(output_csv) == 0:  # Check if file is empty, if so write header
                writer.writeheader()
            writer.writerow(article_details)

        # Mark this URL as processed
        processed_urls.add(url)

        # Wait a moment before processing the next article (optional)
        time.sleep(2)  # Sleep for 2 seconds between article processing to avoid hitting the server too hard

# Function to continuously fetch new articles and immediately process each one
def continuously_fetch_data(base_url, input_csv, output_csv, processed_urls):
    while True:
        print(f"Checking for new articles at: {base_url}")
        articles = extract_articles(base_url)

        # Deduplicate articles
        new_articles = [article for article in articles if article['article site'] not in processed_urls]

        if new_articles:
            print(f"Found {len(new_articles)} new articles. Saving to {input_csv}")
            save_to_csv(new_articles, input_csv)

            # Process the articles from the CSV file and save their details
            process_articles_from_csv(input_csv, output_csv, processed_urls)
        else:
            print("No new articles found. Retrying after a delay.")

        # Sleep for a bit before checking for new articles
        time.sleep(60)  # Wait for 60 seconds before scraping again

# Initialize a set to keep track of processed URLs
processed_urls = set()


# CSV file paths
input_csv = "/content/infinitnews.ai/output/tribune_scraped_data/9tribune_articlesauto.csv"  # Input CSV path
output_csv = "/content/infinitnews.ai/output/tribune_scraped_data/9ai_article_details.csv"  # Output CSV path

# Start the article scraping and processing tasks in parallel using threading
scraping_thread = threading.Thread(target=continuously_fetch_data, args=(base_url, input_csv, output_csv, processed_urls))
scraping_thread.start()

# Join threads to keep the main program running
scraping_thread.join()

Checking for new articles at: https://tribune.com.pk/latest
Found 10 new articles. Saving to /content/infinitnews.ai/output/tribune_scraped_data/9tribune_articlesauto.csv
Processing article: https://tribune.com.pk/story/2511752/drake-takes-shots-at-kendrick-lamar-during-surprise-kick-stream-with-xqc
Processing article: https://tribune.com.pk/story/2511751/im-a-celebrity-star-coleen-rooney-recalls-white-house-visit-and-donald-trumps-controversial-remark
Processing article: https://tribune.com.pk/story/2511749/emma-mckeon-bows-out-a-legendary-swim-career-ends
Processing article: https://tribune.com.pk/story/2511748/taiwans-baseball-team-makes-history
Processing article: https://tribune.com.pk/story/2511747/andy-samberg-praises-bts-jungkooks-standing-next-to-you-fans-celebrate-global-admiration
Processing article: https://tribune.com.pk/story/2511746/kurram-rivals-agree-to-cease-fire-for-7-days
Processing article: https://tribune.com.pk/story/2511745/three-more-polio-cases-confirmed-1
Pro

KeyboardInterrupt: 

#**Article Details and content writing**

In [4]:
import os
import pandas as pd
import re
import logging
import pyttsx3
import os
import logging
from datetime import datetime
from gtts import gTTS
from groq import Groq
from concurrent.futures import ThreadPoolExecutor, as_completed

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[logging.FileHandler("automation.log"), logging.StreamHandler()],
)

# Set the API key as an environment variable
os.environ["GROQ_API_KEY"] = "put_your_groq_api_key"  # Replace with your actual API key

# Initialize Groq client
client = Groq()

# Paths
DATASET_PATH = "/content/infinitnews.ai/output/tribune_scraped_data/9ai_article_details.csv"
OUTPUT_PATH = f"/content/infinitnews.ai/output/ai_gen_articles/output_900{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
AUDIO_OUTPUT_DIR = "/content/infinitnews.ai/output/audiooutput/1audio_files"

# Ensure the audio output directory exists
os.makedirs(AUDIO_OUTPUT_DIR, exist_ok=True)

# Regex pattern for extracting titles
TITLE_PATTERN = r"^\*\*(.+?)\*\*"

# Groq API helpers
def generate_news_speech(article):
    try:
        completion = client.chat.completions.create(
            model="llama3-8b-8192",
            messages=[
                {
                    "role": "system",
                    "content": (
                        "You are a news anchor for InfinitNews preparing for a live broadcast. "
                        "Your task is to create a concise, clear, and engaging summary based on the following article. "
                        "always start with Good evening, and welcome to InfinitNews, The speech should focus on the key points, delivering accurate and factual information in a way that captures the audience’s attention. "
                        "Maintain a formal yet lively tone with smooth transitions between points, as if presenting live on air. "
                        "Keep the language simple and avoid technical jargon, ensuring the content is accessible to a wide audience. "
                        "Your goal is to deliver a polished, professional, and informative segment that is both engaging and easy to follow. give the whole news under 150 characters"
                    ),
                },
                {"role": "user", "content": article},
            ],
            temperature=0.8,
            max_tokens=300,
            top_p=0.9,
        )
        return completion.choices[0].message.content.strip()
    except Exception as e:
        logging.error(f"Error generating speech: {e}")
        return None

def generate_article(content):
    try:
        completion = client.chat.completions.create(
            model="llama3-8b-8192",  # Adjust model name if needed
            messages=[
                {
                    "role": "system",
                    "content": (
                        "Act as a psychological expert, seo expert & news article writer use a informative tone. Your task is to humanize and enhance "
                        "the provided news article by presenting a clear while incorporating key "
                        "details and insights. Ensure the explanation is objective, accurate, and provides a broader "
                        "understanding of the topic make it persuasive and Focus on presenting the news in an engaging yet professional manner ,make sure its not recognized as a ai written article"
                        "write in a format that the important information is clearly stated in the starting, use such a tone to lure in the reader strategically and "
                        "in a way that persuades the reader. Wherever possible, Give a seo optimized clear and compelling title thats formal and persuasades the reader "
                        "cite multiple sources to back up key points, include all the backlinks, and ensure accuracy. write more then 2000 words "
                        "in the end give a disclaimer that this content is AI-Generated News Powered by infinitdigitals.io Strictly for Research Purposes."
                    ),
                },
                {"role": "user", "content": content},
            ],
            temperature=0.7,
            max_tokens=2500,
            top_p=1,
        )
        return completion.choices[0].message.content.strip()
    except Exception as e:
        logging.error(f"Error generating article: {e}")
        return None

def generate_prompt(article):
    try:
        completion = client.chat.completions.create(
            model="llama3-8b-8192",  # Adjust model name if needed
            messages=[
                {
                    "role": "system",
                    "content": (
                        "You are an AI prompt engineer specializing in crafting highly descriptive prompts for GAN image generation. "
                        "Based on the given article, create a prompt that includes setting, style (e.g., realism, Surrealism, imaginative,), mood, objects, aestheticness, set the scenery like premium cinematic film "
                        "characters, colors, and artistic influences. Ensure the description inspires unique, visually engaging, ensure the aestheticness , and relevence and high-quality images using detailed description. "
                        "Do not repeat the title verbatim; use it as inspiration to craft a visually compelling prompt description & dont include any disclaimer example :: Here's a prompt that captures the essence of the article or any disclaimerother in starting, middle or end"
                    ),
                },
                {"role": "user", "content": article},
            ],
            temperature=0.8,
            max_tokens=300,
            top_p=0.9,
        )
        return completion.choices[0].message.content.strip()
    except Exception as e:
        logging.error(f"Error generating prompt: {e}")
        return None

# Updated TTS Function using gTTS
def generate_audio(text, index):
    try:
        if not text or pd.isna(text):
            return None
        file_name = f"news_speech_{index + 1}.mp3"
        audio_path = os.path.join(AUDIO_OUTPUT_DIR, file_name)
        tts = gTTS(text=text, lang="en")
        tts.save(audio_path)
        logging.info(f"Audio file saved at: {audio_path}")
        return audio_path
    except Exception as e:
        logging.error(f"Error generating audio for index {index}: {e}")
        return None


# Utility functions
def extract_title(text):
    if pd.isna(text):
        return None
    match = re.search(TITLE_PATTERN, text, re.MULTILINE)
    return match.group(1) if match else None

def process_in_parallel(data, func):
    results = [None] * len(data)
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_index = {executor.submit(func, item): idx for idx, item in enumerate(data)}
        for future in as_completed(future_to_index):
            idx = future_to_index[future]
            try:
                results[idx] = future.result()
            except Exception as e:
                logging.error(f"Error processing item at index {idx}: {e}")
    return results

def text_to_speech(text, filename):
    try:
        # Initialize TTS engine within the function
        tts_engine = pyttsx3.init()
        tts_engine.save_to_file(text, filename)
        tts_engine.runAndWait()
        logging.info(f"TTS saved to {filename}")
    except Exception as e:
        logging.error(f"Error during TTS conversion: {e}")


# Main Automation Function
def automate_pipeline(dataset_path, output_path):
    try:
        # Load dataset
        logging.info("Loading dataset...")
        df = pd.read_csv(dataset_path)

        # Generate AI articles
        logging.info("Generating AI articles...")
        df["ai_gen_articles"] = process_in_parallel(df["content"].tolist(), generate_article)

        # Extract titles
        logging.info("Extracting titles...")
        df["ai_gen_title"] = df["ai_gen_articles"].apply(extract_title)

        # Generate GAN prompts
        logging.info("Generating GAN prompts...")
        df["gan_prompts"] = process_in_parallel(df["ai_gen_title"].tolist(), generate_prompt)

        # Generate news anchor speeches
        logging.info("Generating news speeches...")
        df["news_speech"] = process_in_parallel(df["ai_gen_articles"].tolist(), generate_news_speech)

        # Convert news speeches to audio files
        logging.info("Converting news speeches to audio...")
        df["audio_path"] = process_in_parallel(
            [(text, idx) for idx, text in enumerate(df["news_speech"].tolist())],
            lambda x: generate_audio(x[0], x[1])
        )

        # Save results
        df.to_csv(output_path, index=False)
        logging.info(f"Pipeline completed. Results saved to {output_path}")
    except Exception as e:
        logging.error(f"Pipeline failed: {e}")


        # Save results
        df.to_csv(output_path, index=False)
        logging.info(f"Pipeline completed. Results saved to {output_path}")
    except Exception as e:
        logging.error(f"Pipeline failed: {e}")

# Run the pipeline
automate_pipeline(DATASET_PATH, OUTPUT_PATH)

ERROR:root:Error generating article: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01j9rwka4hed1sbqp503j7dh1t` on tokens per minute (TPM): Limit 30000, Used 29993, Requested 1068. Please try again in 2.121s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
ERROR:root:Error generating article: Error code: 400 - {'error': {'message': "failed to unmarshal JSON: invalid character 'N' looking for beginning of value", 'type': 'invalid_request_error'}}
ERROR:root:Error generating article: Error code: 400 - {'error': {'message': "failed to unmarshal JSON: invalid character 'N' looking for beginning of value", 'type': 'invalid_request_error'}}
ERROR:root:Error generating article: Error code: 400 - {'error': {'message': "failed to unmarshal JSON: invalid character 'N' looking for beginning of value", 'type': 'invalid_request_error'}}
ERROR:root:Error generating a

KeyboardInterrupt: 

# **GAN article picture Generation**

In [6]:
import requests
from PIL import Image
import io
import pandas as pd
import os

# API Configuration
API_URL = "https://api-inference.huggingface.co/models/stabilityai/stable-diffusion-3.5-large-turbo"
headers = {
    "Authorization": "Bearer HF_TOKEN"  # Replace HF_TOKEN with your actual Hugging Face token
}

# Function to query the API
def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    if response.status_code == 200:
        return response.content
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None

# Read the CSV file
csv_path = "/content/infinitnews.ai/output/ai_gen_articles/output_90020241125_060001.csv"
df = pd.read_csv(csv_path)

# Check if the 'ai_gen_title' column exists
if 'gan_prompts' not in df.columns:
    raise ValueError("The column 'gan_prompts' does not exist in the CSV file.")

# Limit processing to the first three rows
titles = df['gan_prompts'].head(3)

# Directory to save images
output_dir = "/content/infinitnews.ai/output/img_output/generated_images"
os.makedirs(output_dir, exist_ok=True)

# Iterate over the first three titles and generate images
for index, title in enumerate(titles):
    print(f"Processing: {title} ({index + 1}/3)")
    payload = {"inputs": title}
    image_bytes = query(payload)

    if image_bytes:
        try:
            # Create the image
            image = Image.open(io.BytesIO(image_bytes))
            # Save the image with a unique name
            filename = os.path.join(output_dir, f"image_6{index + 1}.png")
            image.save(filename)
            print(f"Image saved: {filename}")
        except Exception as e:
            print(f"Error processing title '{title}': {e}")

print("Images for the first three rows processed and saved.")

Processing: Here's a prompt that captures the essence of the article:

"Generate an image of a dark and gritty urban landscape, inspired by the gritty realism of film noir and the intense energy of a hip-hop concert. A figure in the foreground, reminiscent of a premium cinematic character, stands confidently amidst a backdrop of neon-lit cityscapes and graffiti-covered walls. The atmosphere is electric, with a sense of tension and competition in the air, as if the figure is ready to unleash a lyrical barrage. Incorporate bold, vibrant colors, such as deep blues and fiery oranges, to evoke a sense of intensity and passion. The aesthetic should be high-octane, with a focus on dynamic composition and bold brushstrokes, as if the image was ripped straight from the pages of a graphic novel. The overall mood should be one of unapologetic confidence and unbridled creativity, with a nod to the raw energy and unfiltered passion of hip-hop. Consider the artistic influences of Jean-Michel Basquia

# **RAG FOR NEWS RETREIVAL_GRADIO**

In [1]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
from transformers import pipeline
import gradio as gr

# Load and preprocess the dataset
def load_and_preprocess_data(file_path):
    df = pd.read_csv(file_path)
    df['content'] = df['content'].fillna('').str.strip()
    df['headline'] = df['headline'].fillna('').str.strip()
    return df

# Generate embeddings and create a FAISS index
def create_embeddings_and_index(df, model_name='all-MiniLM-L6-v2'):
    # Load the embedding model
    model = SentenceTransformer(model_name)

    # Generate embeddings for content
    embeddings = model.encode(df['content'].tolist(), show_progress_bar=True)
    np.save('embeddings.npy', embeddings)  # Save embeddings for later use

    # Create and populate the FAISS index
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)
    faiss.write_index(index, 'news_index.faiss')  # Save the FAISS index

    return model, index

# Search function
def search(query, model, index, df, top_k=5):
    try:
        query_embedding = model.encode([query])
        distances, indices = index.search(query_embedding, top_k)
        results = []
        for idx in indices[0]:
            results.append(df.iloc[idx])
        return results
    except Exception as e:
        return f"Error during search: {e}"

# Retrieval-Augmented Generation pipeline
def rag_pipeline(query, model, index, df, summarizer, top_k=5):
    retrieved_articles = search(query, model, index, df, top_k)
    if isinstance(retrieved_articles, str):  # Handle search errors
        return retrieved_articles

    summaries = []
    for article in retrieved_articles:
        try:
            summary = summarizer(article['content'], max_length=100, min_length=30, do_sample=False)
            summaries.append((article['headline'], summary[0]['summary_text']))
        except Exception as e:
            summaries.append((article['headline'], f"Error summarizing content: {e}"))
    return summaries

# Main function to execute all steps
def main():
    # File path to the dataset
    file_path = '/content/infinitnews.ai/output/tribune_scraped_data/9ai_article_details.csv'

    # Load and preprocess the data
    df = load_and_preprocess_data(file_path)

    # Create embeddings and FAISS index
    model, index = create_embeddings_and_index(df)

    # Load summarization model
    summarizer = pipeline('summarization')

    # Build Gradio interface
    def query_interface(user_query):
        results = rag_pipeline(user_query, model, index, df, summarizer)
        return {headline: summary for headline, summary in results}

    iface = gr.Interface(
        fn=query_interface,
        inputs=gr.Textbox(label="Enter your query"),
        outputs=gr.JSON(label="Relevant Summaries"),
        title="Robust RAG Application"
    )
    iface.launch()

# Execute the main function
if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://97918c69520d2c764f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
